<a href="https://colab.research.google.com/github/paulowoicho/msc_project/blob/master/Extractive_Results_with_Coreference_Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install rouge
!pip install neuralcoref==4.0
!pip install spacy==2.1.0 #notebook crashes on other versions
!python -m spacy download en

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.6MB/s 
     |████████████████████████████████| 3.0MB 20.6MB/s 
     |████████████████████████████████| 890kB 30.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=428ca3aac86555c4218528661afdc679cc58d7162f1b9a7ff790a7cfc551ffa6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 296kB 2.8MB/s 
     |████████████████████████████████| 27.7MB 114kB/s 
     |████████████████████████████████| 3.2MB 55.7MB/s 
     |████████████████████████████████| 2.1MB 48.4MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully un

# Data Import/Preparation

In [2]:
import pandas as pd
reference_results = pd.read_csv('150_gold.csv')

In [3]:
reference_results.head(2)

,episode_id,episode_name,episode_description,transcript,textrank_summary,lexrank_summary,lsa_summary,semi_supervised,supervised,extractive,t5_abstractive,first_five
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful re...,"You cannot scummy hashtag wisely, or the peopl...",--- This episode is sponsored by Anchor: The ...,Hello everybody. What's going on in this Jess...
1,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,A Public Service Announcement from the mrbrown...,Don't be a silent victim of crime. a parody fr...,"Ladies, have you been molested don't be a vic...",Have someone escort you home when it's late ab...,But you can touch his life. But you can touch ...,If you only touched you in minor ways never mi...,If you have been molested call the police sho...,"If you have been molested, here are some of t...","Ladies, have you been molested don't be a vict...","Ladies, have you been molested? Don't be a sil...","Ladies, have you been molested don't be a vic..."


#Coreference Resolution 

In [4]:
coref_summaries = reference_results[['episode_id', 'transcript']]
coref_summaries.head(2)

,episode_id,transcript
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Hello everybody. What's going on in this Jess...
1,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,"Ladies, have you been molested don't be a vic..."


In [5]:
import spacy
import neuralcoref
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

def coref_resolution(transcript):
  doc = nlp(transcript)
  return doc._.coref_resolved

100%|██████████| 40155833/40155833 [00:00<00:00, 75167224.15B/s]


In [6]:
coref_summaries['coref_transcript'] = coref_summaries.apply(lambda row: coref_resolution(row['transcript']), axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
coref_summaries.iloc[1]['coref_transcript']

" Ladies, have you been molested don't be a victim of crime here are some of the things you can do. If you have been molested call the police shout for help and call nine nine nine. Don't be a silent victim avoid walking through secluded areas alone.  Have someone escort you home when it's late above all try your best not to be molested by a culprit from a good University who has good academic results and the potential to excel in life. After all we don't want to affect his future in any way. If you only touched you in minor ways never mind love especially if there had been no intrusion into private parts besides,  a culprit might be graduating in the year or two and a culprit's getting help and a culprit says a culprit wants to change and a culprit says a culprit has learned a culprit lesson. So protect the futures of molesters who can contribute to society so much more than you can spare society from the psychological harm and a lifetime of stigma. society need your help to make it h

In [8]:
coref_summaries.iloc[1]['transcript']

" Ladies, have you been molested don't be a victim of crime here are some of the things you can do. If you have been molested call the police shout for help and call nine nine nine. Don't be a silent victim avoid walking through secluded areas alone.  Have someone escort you home when it's late above all try your best not to be molested by a culprit from a good University who has good academic results and the potential to excel in life. After all we don't want to affect his future in any way. If you only touched you in minor ways never mind love especially if there had been no intrusion into private parts besides,  The culprit might be graduating in the year or two and he's getting help and he says he wants to change and he says he has learned his lesson. So protect the futures of molesters who can contribute to society so much more than you can spare them from the psychological harm and a lifetime of stigma. They need your help to make it happen. Remember he only touched your body.  B

In [9]:
#forgot this column
coref_summaries['episode_description'] = reference_results['episode_description']

coref_summaries.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,episode_id,transcript,coref_transcript,episode_description
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Hello everybody. What's going on in this Jess...,Hello everybody. What's going on in this Jess...,If you want to start mastering recruiting whic...
1,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,"Ladies, have you been molested don't be a vic...","Ladies, have you been molested don't be a vic...",Don't be a silent victim of crime. a parody fr...


#How is it on the baselines?

In [10]:
!pip install sumy

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 10.1MB 8.5MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=f5ddeb58736c683da7bb48b4223a2bdbd5c5748828e0bf9b74d3859ff712c736
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21684 sha256=fb00e4b9ae0c62f5305f85242367d400c4081cc1e71d86f631fa1072ab93183c
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
Successfully built pycountry breadability


In [11]:
!python sumy_example.py #script that handles sumy implementation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
from sumy_example import summarize

coref_summaries['textrank_summary'] = coref_summaries.apply(lambda row: summarize(row['coref_transcript'], 'text_rank'), axis=1)
coref_summaries['lexrank_summary'] = coref_summaries.apply(lambda row: summarize(row['coref_transcript'], 'lex_rank'), axis=1)
coref_summaries['lsa_summary'] = coref_summaries.apply(lambda row: summarize(row['coref_transcript'], 'lsa'), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (40) is lower than number of sentences (157). LSA algorithm may not work prope

#Bert + Kmeans with coref

In [13]:
!git clone https://github.com/paulowoicho/bert-extractive-summarizer.git
!mv bert-extractive-summarizer summarizer

Cloning into 'bert-extractive-summarizer'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 378 (delta 0), reused 3 (delta 0), pack-reused 374
Receiving objects: 100% (378/378), 85.48 KiB | 3.42 MiB/s, done.
Resolving deltas: 100% (215/215), done.


In [14]:
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler
from transformers import *

handler = CoreferenceHandler(greedyness=.4)

extractive_model = Summarizer(sentence_handler=handler)

custom_config = AutoConfig.from_pretrained("SpanBERT/spanbert-base-cased")
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained("SpanBERT/spanbert-base-cased")
custom_model = AutoModel.from_pretrained("SpanBERT/spanbert-base-cased", config=custom_config)

spanbert_model = Summarizer(custom_model = custom_model, sentence_handler=handler, custom_tokenizer=custom_tokenizer)

def extractive_summary(transcript):
  result = extractive_model(transcript, min_length=60, ratio=0.01)
  return ''.join(result)

def span_bert(transcript):
  result = spanbert_model(transcript, min_length=60, ratio=0.01)
  return ''.join(result)

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
coref_summaries['extractive'] = coref_summaries.apply(lambda row: extractive_summary(row['transcript']), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
coref_summaries['spanbert_extractive'] = coref_summaries.apply(lambda row: span_bert(row['transcript']), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#Evaluation

In [17]:
from rouge import Rouge
rouge = Rouge()

textrank = rouge.get_scores(coref_summaries['episode_description'], coref_summaries['textrank_summary'], avg=True)
lexrank = rouge.get_scores(coref_summaries['episode_description'], coref_summaries['lexrank_summary'], avg=True)
lsa = rouge.get_scores(coref_summaries['episode_description'], coref_summaries['lsa_summary'], avg=True)
extractive = rouge.get_scores(coref_summaries['episode_description'], coref_summaries['extractive'], avg=True)
spanbert_extractive = rouge.get_scores(coref_summaries['episode_description'], coref_summaries['spanbert_extractive'], avg=True)


In [18]:
row_names = ['textrank', 'lexrank', 'lsa', 'extractive', 'spanbert_extractive']
rouge1_scores = [textrank['rouge-1']['f'], lexrank['rouge-1']['f'], lsa['rouge-1']['f'], extractive['rouge-1']['f'], spanbert_extractive['rouge-1']['f']]

rouge1_df = pd.DataFrame({'Rouge1-F':rouge1_scores}, index=row_names)
rouge1_df

,Rouge1-F
textrank,0.105327
lexrank,0.123332
lsa,0.133997
extractive,0.146966
spanbert_extractive,0.145516


In [19]:
rouge2_scores = [textrank['rouge-2']['f'], lexrank['rouge-2']['f'], lsa['rouge-2']['f'], extractive['rouge-2']['f'], spanbert_extractive['rouge-2']['f']]

rouge2_df = pd.DataFrame({'Rouge2-F':rouge2_scores}, index=row_names)
rouge2_df

,Rouge2-F
textrank,0.008477
lexrank,0.013459
lsa,0.012981
extractive,0.023059
spanbert_extractive,0.026931


In [20]:
rougel_scores = [textrank['rouge-l']['f'], lexrank['rouge-l']['f'], lsa['rouge-l']['f'], extractive['rouge-l']['f'], spanbert_extractive['rouge-l']['f']]

rougel_df = pd.DataFrame({'RougeL-F':rougel_scores}, index=row_names)
rougel_df

,RougeL-F
textrank,0.100659
lexrank,0.108949
lsa,0.112228
extractive,0.123626
spanbert_extractive,0.125482


In [21]:
rouge_scores = rouge1_df.merge(rouge2_df, left_index=True, right_index = True)
rouge_scores = rouge_scores.merge(rougel_df, left_index=True, right_index = True)

In [22]:
rouge_scores

,Rouge1-F,Rouge2-F,RougeL-F
textrank,0.105327,0.008477,0.100659
lexrank,0.123332,0.013459,0.108949
lsa,0.133997,0.012981,0.112228
extractive,0.146966,0.023059,0.123626
spanbert_extractive,0.145516,0.026931,0.125482
